# JS09 - CI/CD Tutorial

## Overview

In this tutorial we will build a simple CI/CD script for GDN plateform

## Pre-requisite

Lets Assume 
- you have already made a tenant account, and have a username and password
- you have installed the jsc8 drivers as explained in section 01
- you have generated an API Key as explained in section 01


In [ ]:
#/* run this once to install javascript kernal and jsc8 in google colab, then reload, and then skip this
!npm install jsc8
!npm install -g --unsafe-perm ijavascript
!ijsinstall --install=global  # */

## 1. Importing Libraries & Define Variables

The first step is to import the libraries we need and define the variables we will be using in this tutorial. This is also the right place to add your GDN login credentials. i.e. your email and password. You will also need to make sure you have specified the correct federation URL. In this example it is "gdn.paas.macrometa.io" and we are using the default geo fabric "_system".

In [ ]:
const fed_url = "https://gdn.pass.macrometa.io";
const email = "email"; // <-- Email goes here
const password = "password"; // <-- password goes here
const geo_fabric = "_system";

const COLLECTION_TYPE = {
  DOCUMENT: "document",
  KV: "graph",
};

const fabricList = ["cicd-fabric"];

const collectionList = [
  {
    name: "Doc1",
    type: COLLECTION_TYPE.DOCUMENT,
    isEdge: false,
    hasStream: true,
    noOfInsertOperation: 100,
    noOfUpdateOperation: 1,
    noOfDeleteOperations: 100,
  },
  {
    name: "Doc2",
    type: COLLECTION_TYPE.DOCUMENT,
    isEdge: false,
    hasStream: true,
    noOfInsertOperation: 100,
    noOfUpdateOperation: 1,
    noOfDeleteOperations: 100,
  },
  { name: "KV1", type: COLLECTION_TYPE.KV, hasStream: true },
];

const streamList = [
  {
    name: "gstream",
    isLocal: false,
    noOfInsertOperation: 10,
    noOfUpdateOperation: 0,
    noOfDeleteOperations: 0,
  },
  {
    name: "lstream",
    isLocal: true,
    noOfInsertOperation: 10,
    noOfUpdateOperation: 0,
    noOfDeleteOperations: 0,
  },
];

const restqlList = {
  insert_data: {
    name: "insertRecord",
    value: `FOR i IN 1..100 
                INSERT {
                    "firstname":CONCAT("Halie", TO_STRING(i)),
                    "lastname":CONCAT("Linkie", TO_STRING(i)),
                    "email":CONCAT("hlinkie0", TO_STRING(i),"@irs.gov"),
                    "zipcode": CONCAT("2950-53", TO_STRING(i))
                } INTO COLLECTION_NAME`,
    parameter: {},
  },
  get_data: {
    name: "getRecords",
    value: `FOR doc IN COLLECTION_NAME RETURN doc`,
  },
  update_data: {
    name: "updateRecord",
    value: `FOR doc IN COLLECTION_NAME 
      filter doc.email == 'hlinkie05@irs.gov'
      UPDATE { _key:doc._key,  \"lastname\": \"cena\" }
        IN COLLECTION_NAME`,
  },
  get_count: {
    name: "countRecords",
    value: `RETURN COUNT(FOR doc IN COLLECTION_NAME RETURN 1)`,
  },
  delete_data: {
    name: "deleteRecord",
    value: `FOR doc IN COLLECTION_NAME 
          filter doc.email == 'hlinkie03@irs.gov'
          REMOVE doc 
          IN COLLECTION_NAME `,
  },
  delete_all_data: {
    name: "deleteAllRecord",
    value: `FOR doc IN COLLECTION_NAME
        REMOVE doc 
        IN COLLECTION_NAME `,
  },
};

const graphList = [
  {
    name: "social",
    edgeDefinitions: [
      {
        collection: "relation",
        from: ["female", "male"],
        to: ["female", "male"],
      },
    ],
  },
  { name: "children" },
];

const streamWorkerList = [
  {
    name: "MockHeartRateDataGenerator",
    definition: `
    @App:name("MockHeartRateDataGenerator")
    @App:qlVersion("2")

    CREATE TRIGGER HeartRateDataGeneratorTrigger WITH ( interval = 10 sec );

    CREATE TABLE HeartRates (name string, bpm int);


    -- Note: Generating random bpm and name 
    @info(name = 'ConsumeProcessedData')
    INSERT INTO HeartRates
    SELECT 
    js:eval("['Vasili', 'Rivalee', 'Betty', 'Jennifer', 'Alane', 'Sarena', 'Bruno', 'Carolee', 'Emmott', 'Andre'][Math.floor(Math.random() * 10)]","string") as name,
    js:eval("Math.floor(Math.random() * 40) + 40","int") as bpm
    FROM HeartRateDataGeneratorTrigger;
    `,
  },
];

const webSocketData = {};

const webSocketList = [];

let currentFabric = "";

const DOCUMENT_OPERATIONS = {
  UPDATE: "UPDATE",
  INSERT: "INSERT",
  DELETE: "DELETE",
};

const info = {
  noOfCollection: 0,
  noOfStreamWorker: 0,
  noOfQueryWorker: 0,
  noOfStream: 0,
  noOfFabric: 0,
  noOfGraph: 0,
};

let executeCiCDPipeLine;

## 2. Connecting to GDN

Now that we have imported the required libraries and added our login details, we can connect to GDN. Do this by running the cell bellow.

You will see the cell output reflect a successful connection. If not go back to the first step and check the details you entered.

In [ ]:
const jsc8 = require("jsc8");

// ----- simple way  -----
const client = new jsc8(fed_url);

client.useFabric(geo_fabric);
client
  .login(email, password)
  .then((result) => console.log("Login successfully", result))
  .catch((err) => console.error("Error while login", err.message));

## 3. Creating Helper Functions

Helper functions are basic building blocks for the CI/CD pipeline. These functions are responsible for creating/deleting collection, streams, query worker, stream worker, etc. based on the list provied in the section 1. 

In [ ]:
// Logger function
const log = (message) => {
  console.info(`[${new Date()}]: ${message}`);
};

// Generate the random numbers
const getRandomNumber = (min, max) => {
  return Math.floor(Math.random() * (max - min + 1)) + min;
};

// Create the document collection
const createDocumentCollection = async (
  collectionName,
  hasStream = true,
  isEdge = false
) => {
  const hasCollection = await client.hasCollection(collectionName);
  if (hasCollection) return;
  await client.createCollection(collectionName, { stream: hasStream }, isEdge);
  info.noOfCollection += 1;
};

// Create the kv collection
const createKvCollection = async (collectionName, hasStream = true) => {
  const hasCollection = await client.hasCollection(collectionName);
  if (hasCollection) return;
  await client.createKVCollection(collectionName, { stream: hasStream });
  info.noOfCollection += 1;
};

// Create the query workers
const createRestQL = async (resetql, restqlList) => {
  try {
    for (const collection of collectionList) {
      const queryWorkerName = `${collection.name}_${resetql.name.toString()}`;
      if (
        collection.type != COLLECTION_TYPE.DOCUMENT ||
        restqlList.includes(queryWorkerName)
      )
        continue;
      await client.createRestql(
        queryWorkerName,
        resetql.value.toString().replaceAll("COLLECTION_NAME", collection.name),
        resetql.parameter
      );
    }
  } catch (error) {
    error.message =
      `Error while creating restql ${resetql.name} : -- ` + error.message;
    throw error;
  }
};

// Create the GeoFabric
const createFabric = async (fabricName, fabricList, localDcName, allDcList) => {
  let globalDcList = [
    allDcList[getRandomNumber(0, allDcList.length - 1)],
    allDcList[getRandomNumber(0, allDcList.length - 1)],
  ].filter((dataCenter) => dataCenter);
  const dcList = [...new Set([...globalDcList, localDcName])];
  if (fabricList.includes(fabricName)) return;
  await client.createFabric(fabricName, [], {
    dcList: dcList,
  });
  info.noOfFabric += 1;
};

// Create the graph
const crateGraph = async (graphName, graphDetails = null) => {
  const hasGraph = await client.hasGraph(graphName);
  if (hasGraph) return;
  await client.createGraph(graphName, graphDetails);
  info.noOfGraph += 1;
};

// Create the stream
const createStream = async (streamName, isLocal) => {
  const isStreamExist = await client.hasStream(streamName, isLocal);
  if (isStreamExist) return;
  await client.createStream(streamName, isLocal);
  info.noOfStream += 1;
};

// Create the stream subscriber
const createSubscriber = async (
  streamName,
  isCollection = true,
  isLocal = true
) => {
  try {
    const wsConnection = await client.createStreamReader(
      streamName,
      streamName,
      isLocal,
      isCollection,
      client._connection._urls[0].replace("https://api-", "")
    );
    // await client.deleteStreamSubscription(streamName, streamName, isLocal, isCollection);

    wsConnection.on("open", () => {
      log(`Connection open for ${streamName}`);
      webSocketData[streamName] = {};
      webSocketData[streamName][DOCUMENT_OPERATIONS.UPDATE] = [];
      webSocketData[streamName][DOCUMENT_OPERATIONS.INSERT] = [];
      webSocketData[streamName][DOCUMENT_OPERATIONS.DELETE] = [];
    });
    wsConnection.on("close", () => log(`Connection close for ${streamName}`));
    wsConnection.on("error", (error) => log(error));

    wsConnection.on("message", (event) => {
      const encodedMessage = JSON.parse(event).payload;
      let data = {};
      const decodedMessage = atob(encodedMessage);

      if (decodedMessage.length !== 0) {
        data = JSON.parse(decodedMessage);
      }
      const oldDocumentData = webSocketData[streamName];
      switch (JSON.parse(event).properties.op) {
        case DOCUMENT_OPERATIONS.UPDATE:
          oldDocumentData[DOCUMENT_OPERATIONS.UPDATE].push(data);
          break;
        case DOCUMENT_OPERATIONS.DELETE:
          oldDocumentData[DOCUMENT_OPERATIONS.DELETE].push(data);
          break;
        default:
          oldDocumentData[DOCUMENT_OPERATIONS.INSERT].push(data);
      }
      wsConnection.send(
        JSON.stringify({ messageId: JSON.parse(event).messageId })
      );
    });
    webSocketList.push(wsConnection);
  } catch (error) {
    error.message =
      `Error while creating subscriber ${streamName} : -- ` + error.message;
    throw error;
  }
};

// Create the stream publisher
const createPublisher = async (streamName, isLocal, isCollection) => {
  const producer = await client.createStreamProducer(
    streamName,
    isLocal,
    isCollection
  );
  producer.on("open", () => {
    // If you message is an object, convert the obj to string.
    // e.g. const message = JSON.stringify({message:'Hello World'});
    for (let i = 0; i < 10; i++) {
      const message = {
        firstname: `Halie${i}`,
        lastname: `Linkie${i}`,
        email: `hlinkie0${i}@irs.gov`,
        zipcode: `2950-53${i}`,
      };
      const payloadObj = {
        payload: Buffer.from(JSON.stringify(message)).toString("base64"),
      };
      producer.send(JSON.stringify(payloadObj));
    }
  });
};

// Create the stream based on provided stream list in streamList
const createStreams = async () => {
  const promiseList = streamList.map((stream) =>
    createStream(stream.name, stream.isLocal)
  );

  await Promise.all(promiseList);
};

// Create the stream subscribers based on provided stream list in streamList
const createStreamSubscribers = async () => {
  const promiseList = streamList.map((stream) =>
    createSubscriber(stream.name, false, stream.isLocal)
  );
  await Promise.all(promiseList);
};

// Create the stream publisher based on provided stream list in streamList
const createStreamPublisher = async () => {
  const promiseList = streamList.map((stream) =>
    createPublisher(stream.name, stream.isLocal, false)
  );
  await Promise.all(promiseList);
};

// Create the new fabric
const createNewFabrics = async () => {
  const localDc = await client.getLocalDc();
  const localDcName = localDc["_key"];
  const [allDcList] = await client.getDcList();
  let globalDcList = allDcList.dcInfo
    .map((dc) => dc._key)
    .filter((dcName) => dcName != localDcName);
  const existingFabricList = await client.listFabrics();

  const promiseList = fabricList.map((fabric) =>
    createFabric(fabric, existingFabricList, localDcName, globalDcList)
  );
  await Promise.all(promiseList);

  currentFabric = fabricList[getRandomNumber(0, fabricList.length - 1)];
  client.useFabric(currentFabric);
};

// Create the collections based on provided collection list in collectionList
const createCollections = async () => {
  const promiseList = collectionList.map((collection) => {
    let func = createDocumentCollection;
    if (collection.type === COLLECTION_TYPE.KV) func = createKvCollection;
    return func(collection.name, collection.hasStream, collection.isEdge);
  });

  await Promise.all(promiseList);
};

// Create the query workers based on provided query worker list in restqlList
const createRestQLs = async () => {
  for (const restqlKey of Object.keys(restqlList)) {
    let existingRestqlList = await client.getRestqls();
    existingRestqlList = existingRestqlList.result.map((restql) => restql.name);
    await createRestQL(restqlList[restqlKey], existingRestqlList);
    info.noOfQueryWorker += 1;
  }
};

// Create the subscriber based on provided collection list in collectionList
const createCollectionsSubscribers = async () => {
  const promiseList = collectionList
    .filter((collection) => collection.type === COLLECTION_TYPE.DOCUMENT)
    .map((collection) => createSubscriber(collection.name));
  await Promise.all(promiseList);
};

// Create the graphs based on provided graph list in graphList
const createGraphs = async () => {
  const promiseList = graphList.map((graph) =>
    crateGraph(graph.name, { edgeDefinitions: graph.edgeDefinitions })
  );

  await Promise.all(promiseList);
};

// Create the stream worker based on provided stream list in streamAppList
const createStreamWorkers = async () => {
  const promiseList = [];
  let streamAppList = await client.retrieveStreamApp();
  streamAppList = streamAppList.streamApps.map((streamApp) => streamApp.name);
  streamWorkerList.forEach((streamWorker) => {
    if (!streamAppList.includes(streamWorker.name)) {
      promiseList.push(
        client
          .createStreamApp([], streamWorker.definition)
          .then((data) => (info.noOfStreamWorker += 1))
      );
    }
  });
  await Promise.all(promiseList);
};

// Activate the stream worker based on provided stream list in streamWorkerList
const activateStreamWorkers = async (isActive = true) => {
  const promiseList = [];
  streamWorkerList.forEach((streamWorker) => {
    promiseList.push(client.activateStreamApp(streamWorker.name, isActive));
  });
  await Promise.all(promiseList);
};

// set current fabric to jsc8 client
const useFabric = () => {
  client.useFabric(currentFabric);

  log(`Using fabric ${currentFabric}`);
};

/*
Execute all the query workers provided in the restqlList 
for every document collection present in collectionList 
*/
const executeRestQLs = async () => {
  for (const restql of Object.keys(restqlList)) {
    for (const collection of collectionList) {
      if (collection.type === COLLECTION_TYPE.DOCUMENT) {
        await client.executeRestql(
          `${collection.name}_${restqlList[restql].name.toString()}`,
          restqlList[restql].data
        );
      }
    }
  }
};

// Read the websocket data and verify the number of operations
const verifyStreams = async () => {
  // wait for 5 seconds to receive all the stream data
  await new Promise((resolve) => setTimeout(resolve, 5000));
  streamList.forEach((stream) => {
    const wsData = webSocketData[stream.name];
    if (
      !(
        wsData[DOCUMENT_OPERATIONS.INSERT].length ===
          stream.noOfInsertOperation &&
        wsData[DOCUMENT_OPERATIONS.UPDATE].length ===
          stream.noOfUpdateOperation &&
        wsData[DOCUMENT_OPERATIONS.DELETE].length ===
          stream.noOfDeleteOperations
      )
    )
      throw Error(`Websocket data validation failed for ${stream.name}`);
  });
};

// Validate the collection stream websocket data and operations
const verifyCollectionStream = async () => {
  // wait for 5 seconds to receive all the stream data
  await new Promise((resolve) => setTimeout(resolve, 5000));
  collectionList
    .filter((collection) => collection.type === COLLECTION_TYPE.DOCUMENT)
    .forEach((collection) => {
      const wsData = webSocketData[collection.name];
      if (
        !(
          wsData[DOCUMENT_OPERATIONS.INSERT].length ===
            collection.noOfInsertOperation &&
          wsData[DOCUMENT_OPERATIONS.UPDATE].length ===
            collection.noOfUpdateOperation &&
          wsData[DOCUMENT_OPERATIONS.DELETE].length ===
            collection.noOfDeleteOperations
        )
      )
        throw Error(
          `Websocket data validation failed for collection ${collection.name}`
        );
    });
};

// Delete all the query workers present in the fabric
const deleteRestQLs = async () => {
  const promiseList = [];
  const restqlList = await client.listSavedQueries();
  restqlList.result.forEach((query) => client.deleteRestql(query.name));

  await Promise.all(promiseList);
};

// Delete all the graphs present in the fabric
const deleteGraphs = async () => {
  const graphList = await client.listGraphs();
  const promiseList = graphList.map((graph) =>
    client.deleteGraph(graph.name, true)
  );
  await Promise.all(promiseList);
};

// Delete all the collection present in the fabric
const deleteCollections = async () => {
  const collectionList = await client.listCollections();
  const promiseList = collectionList.map((collection) =>
    client.deleteCollection(collection.name)
  );
  await Promise.all(promiseList);
};

// Delete all the fabric workers present in the fabricList
const deleteFabrics = async () => {
  client.useFabric("_system");
  const promiseList = fabricList.map((fabricName) =>
    client.dropFabric(fabricName)
  );
  await Promise.all(promiseList);
};

// Terminate all the websocket
const clearWebsocket = async () => {
  webSocketList.forEach((webSocket) => {
    webSocket.terminate();
  });
};

// Delete all the streams present in the fabric
const deleteStreams = async () => {
  const streamsList = await client.getStreams();
  const promiseList = streamsList.result.map((stream) =>
    client.stream(stream.topic).deleteStream()
  );
  await Promise.all(promiseList);
};

// Delete all the streams worker present in the fabric
const deleteStreamWorker = async () => {
  const allSteamApps = await client.getAllStreamApps();
  const promiseList = allSteamApps.streamApps.map((streamApp) =>
    client.deleteStreamApp(streamApp.name)
  );
  await Promise.all(promiseList);
};

// It will delete the query-workers, graphs, collections, streams, stream-workers and fabric
const clear = async () => {
  useFabric();
  await deleteRestQLs();
  await deleteGraphs();
  await deleteCollections();
  await deleteStreams();
  await deleteStreamWorker();
  await deleteFabrics();
};

## 4. Create CI/CD Pipeline

Below function contains the list of sub functions.Each subfunction is responsible for either creating or deleting the elements in the GDN. 

Current flow is as below 
- Create fabrics
- Pick any random fabric from created fabric
- Create collections
- Create query worker for each collection 
- Create stream workers 
- Activate the stream workers
- Create the collection subscribers
- Execute the query workers
- Execute the CRUD using import API and truncate the collection. This operation is operformed on each collection 
- Validate the collection streams after query worker and CRUD operation execution with expected output
- Create streams
- Create the stream subscribers
- Create the stream publisher. Publisher is pushing some records in the streams
- Verify the stream output by counting the number of object received in the stream.
- Unpublish the stream worker
- Delete all the queryworkers, all the graphs, all the collections, all the streams, all the stream workers, and delete the number of fabric we created in the starting of program


If you want to customize the CI/CD pipeline then please comment the appropriate steps and modify the CI/CD pipeline.

In [ ]:
// It will execute GDN pipeline in series manner
const executeCiCDPipeLine = async () => {
  
  try {
    log("\n ------- CREATE FABRICS ------");
    await createNewFabrics();
    log("\n ------- CREATED FABRICS ------");

    useFabric();

    log("\n ------- CREATE GEO-REPLICATED COLLECTION  ------");
    await createCollections();
    log("\n ------- CREATED GEO-REPLICATED COLLECTION  ------");

    log("\n ------- CREATE GRAPHS ------");
    await createGraphs();
    log("\n ------- CREATED GRAPHS  ------");

    log("\n ------- CREATE RESTQLS  ------");
    await createRestQLs();
    log("\n ------- CREATED RESTQLS  ------");

    log("\n ------- CREATE STREAM WORKERS  ------");
    await createStreamWorkers();
    log("\n ------- CREATED STREAM WORKERS  ------");

    log("\n ------- PUBLISH STREAM WORKERS  ------");
    await activateStreamWorkers();
    log("\n ------- PUBLISH STREAM WORKERS  ------");

    log("\n ------- CREATE COLLECTION SUBSCRIBER  ------");
    await createCollectionsSubscribers();
    log("\n ------- CREATED COLLECTION SUBSCRIBER  ------");

    log("\n ------- EXECUTE CRUD USING RESETQL ------");
    await executeRestQLs();
    log("\n ------- EXECUTED CRUD USING RESETQL ------");

    log("\n ------- VERIFY COLLECTION STREAM ------");
    await verifyCollectionStream();
    log("\n ------- VERIFIED COLLECTION STREAM ------");

    log("\n ------- CREATE STREAM ------");
    await createStreams();
    log("\n ------- CREATED STREAM ------");

    log("\n ------- SUBSCRIBE STREAM ------");
    await createStreamSubscribers();
    log("\n ------- SUBSCRIBED STREAM ------");

    log("\n ------- CREATE STREAM PRODUCER ------");
    await createStreamPublisher();
    log("\n ------- CREATED STREAM PRODUCER ------");

    log("\n ------- VERIFY STREAM ------");
    await verifyStreams();
    log("\n ------- VERIFY STREAM ------");

    log("\n ------- UNPUBLISH STREAM WORKERS  ------");
    await activateStreamWorkers(false);
    log("\n ------- UNPUBLISH STREAM WORKERS  ------");

    // clearing everything from previous runs
    log("\n ------- CLEARING ------");
    await clear();
    log("\n ------- CLEARED  ------");

    log(`Info: ${JSON.stringify(info)}`);
  } catch (error) {
    console.log(error.message);
  } finally {
    clearWebsocket();
  }
};

## 5. Execute CI/CD script

Below function is executing the CI/CD pipeline. 

In [ ]:
// Execute the CI/CD pipeline
executeCiCDPipeLine();

## Section Completed!

Congratulations! you have completed this tutorial.